## Setup

In [1]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [3]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [4]:
# Set item range table name
table_GM = 'GM_item_range' + datetime.today().strftime('%Y_%m_%d')

In [5]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table_GM,
                'filter' : "AND item_sub_department_name = '400 GROWING MEDIA' "})

snowflake_tools - 2025-07-07 20:08:07.032382+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-07-07 20:08:12.757573+10:00 - Saved token with '2025-07-08 00:08:12+10:00' expiry...


## Demographic Segments (AU)

### Overall index

In [6]:
ds = con.read_sql_file_into_pandas(
    file="sql/demographic-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 GROWING MEDIA'"
    }
)

In [18]:
print(f"Total sales: ${ds['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(ds['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(ds['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $160,691,891.00
Total quantity: 18,538,506
Total customers: 2,592,008


In [8]:
ds[["DEMOGRAPHIC_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,DEMOGRAPHIC_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Homeowner older kids,12658140.01,0.078773,0.83
1,Homeowner younger kids,21650028.46,0.134730,0.72
2,Older homeowner,87747275.73,0.546059,1.31
3,Renter no kids,9191033.47,0.057197,0.65
4,Renter with kids,5073012.15,0.031570,0.52
5,Unclassifiable,1434624.35,0.008928,1.31
6,Younger homeowner,22937776.83,0.142744,0.99


$161M in sales, 54% comes from Older homeowner
- Older homeowner over indexed - loves gardening, have large garden
- Homwowner younger kids and renters under indexed - no garden?

### Class index

In [19]:
GM_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [22]:
summary = (
    GM_ds[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 COIR MULCH BLOCK HYDRO,1155603.63,60511.0,30355,0.007191,0.003264,0.006839
7,500 COIR POTTING MIX BLOCK,708514.12,84519.0,49384,0.004409,0.004559,0.011126
14,500 COMPOSTS SOIL CONDITIONERS,29642027.43,4166671.0,881094,0.184465,0.224758,0.198498
21,500 MANURE,5803884.42,1101361.0,293716,0.036118,0.059409,0.066170
28,500 MULCHES,39012103.21,3291411.0,742063,0.242776,0.177545,0.167177
35,500 POTTING MIXES,76869003.63,8938231.0,1948306,0.478363,0.482144,0.438926
42,500 SMALL BAG POTTING MIX,7500754.56,895802.0,493880,0.046678,0.048321,0.111264


In [20]:
GM_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 COIR MULCH BLOCK HYDRO,Homeowner older kids,100865.04,0.087283,1.108041
1,500 COIR MULCH BLOCK HYDRO,Homeowner younger kids,180888.60,0.156532,1.161817
2,500 COIR MULCH BLOCK HYDRO,Older homeowner,543161.02,0.470024,0.860756
3,500 COIR MULCH BLOCK HYDRO,Renter no kids,84448.50,0.073077,1.277652
4,500 COIR MULCH BLOCK HYDRO,Renter with kids,42079.39,0.036413,1.153423
5,500 COIR MULCH BLOCK HYDRO,Unclassifiable,8000.41,0.006923,0.775459
6,500 COIR MULCH BLOCK HYDRO,Younger homeowner,196160.67,0.169747,1.189175
7,500 COIR POTTING MIX BLOCK,Homeowner older kids,55588.85,0.078458,0.996009
8,500 COIR POTTING MIX BLOCK,Homeowner younger kids,97895.37,0.138170,1.025532
9,500 COIR POTTING MIX BLOCK,Older homeowner,343730.03,0.485142,0.888442


48% sales comes from 500 POTTING MIXES, 24% from 500 MULCHES, 18% from 500 COMPOSTS SOIL CONDITIONERS, the rest 10% are minimal...	
- Renter no kids under indexed in 500 MANURE, 500 MULCHES
- Renters over indexed in 500 COIR MULCH BLOCK HYDRO, 500 COIR POTTING MIX BLOCK, 500 SMALL BAG POTTING MIX
- Homeowner younger kids over indexed in 500 COMPOSTS SOIL CONDITIONERS

## Commercial Industry Segment (NZ)

### Overall index

In [10]:
cis = con.read_sql_file_into_pandas(
    file="sql/commercial-industry-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 GROWING MEDIA'",
        'country' : 'NZ'
    }
)

In [27]:
print(f"Total sales: ${cis['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(cis['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(cis['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $4,075,711.62
Total quantity: 474,806
Total customers: 34,878


In [23]:
n = 7
top_n = cis.nlargest(n, "RANGE_SALES").copy()
top_n[["INDUSTRY_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,INDUSTRY_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Professional Computer and Scientific Services,672372.09,0.164970,2.00
1,Personal and Other Services,574593.10,0.140980,2.33
2,Rental Hiring and Real Estate Services,329816.04,0.080922,0.93
3,Retail and Wholesale Trade,309192.48,0.075862,1.49
4,Residential Builder,308458.70,0.075682,0.30
5,Manufacturing,294825.51,0.072337,1.40
6,Site Preparation and Development,214062.55,0.052522,0.74


In [12]:
print(f"Top {n} segments - sales share: {top_n['RANGE_SALES_SHARE'].sum():.2%}")
print(f"Top {n} segments - quantity share: {top_n['RANGE_QUANTITY_SHARE'].sum():.2%}")
print(f"Top {n} segments - customer share: {top_n['RANGE_CUSTOMER_SHARE'].sum():.2%}")
print(f"Top {n} segments - size share: {top_n['SEGMENT_SIZE_SHARE'].sum():.2%}")

Top 7 segments - sales share: 66.33%
Top 7 segments - quantity share: 66.39%
Top 7 segments - customer share: 60.23%
Top 7 segments - size share: 41.56%


### Class index

In [13]:
# Get data
GM_is = con.read_sql_file_into_pandas(
    file="sql/2-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': n, 
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [28]:
summary = (
    GM_is[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 COIR POTTING MIX BLOCK,11480.16,1960.0,315,0.004678,0.006865,0.011212
7,500 COMPOSTS SOIL CONDITIONERS,734989.60,104549.0,8786,0.299487,0.366201,0.312714
14,500 MULCHES,393538.65,41765.0,3593,0.160356,0.146289,0.127883
21,500 POTTING MIXES,1200537.67,124722.0,11996,0.489184,0.436861,0.426965
28,500 SMALL BAG POTTING MIX,113616.34,12500.0,3406,0.046295,0.043783,0.121227


In [29]:
GM_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 COIR POTTING MIX BLOCK,Commercial Builder,117.44,0.010230,0.418187
1,500 COIR POTTING MIX BLOCK,Manufacturing,1852.68,0.161381,1.343355
2,500 COIR POTTING MIX BLOCK,Personal and Other Services,2756.83,0.240139,1.025664
3,500 COIR POTTING MIX BLOCK,Professional Computer and Scientific Services,2996.74,0.261036,0.952785
4,500 COIR POTTING MIX BLOCK,Rental Hiring and Real Estate Services,1265.87,0.110266,0.820489
5,500 COIR POTTING MIX BLOCK,Residential Builder,1999.29,0.174152,1.385588
6,500 COIR POTTING MIX BLOCK,Site Preparation and Development,491.31,0.042796,0.490648
7,500 COMPOSTS SOIL CONDITIONERS,Commercial Builder,19449.50,0.026462,1.081758
8,500 COMPOSTS SOIL CONDITIONERS,Manufacturing,84287.44,0.114678,0.954597
9,500 COMPOSTS SOIL CONDITIONERS,Personal and Other Services,149858.35,0.203892,0.870849


## Drop item range table

In [17]:
# # Drop item range table
# con.execute_statement_from_sql_string(
#     statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
#     formatting={'table_name': table_GM}
#     )